In [131]:
import pandas as pd 
import glob as glob
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [132]:
paths = glob.glob(f'./*emission*')

In [133]:
df1 = pd.DataFrame()

for path in paths:
    tmp = pd.read_csv(path)

    df1 = pd.concat([df1, tmp], ignore_index=True)

In [134]:
df1['date'] = pd.to_datetime(df1['start_time'])
df1['date'] = df1['date'].dt.to_period('Y')

In [135]:
df1 = (
    df1.groupby(['source_id', 'source_name', 'source_type', 'iso3_country', 'sector', 'subsector', 'lat', 'lon', 'gas', 'date'])
    ['emissions_quantity']
    .sum()
    .reset_index()
)

In [136]:
df1['geometry'] = df1.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

In [137]:
df1 = gpd.GeoDataFrame(df1, geometry='geometry')
df1 = df1.set_crs(epsg=4326, inplace=True)

In [138]:
city_boundaries = pd.read_csv('./data-1732727290684.csv')

In [139]:
def create_polygon(row):
    return Polygon([
        (row['bbox_west'], row['bbox_north']),  # NW corner
        (row['bbox_east'], row['bbox_north']),  # NE corner
        (row['bbox_east'], row['bbox_south']),  # SE corner
        (row['bbox_west'], row['bbox_south']),  # SW corner
        (row['bbox_west'], row['bbox_north'])   # Close the polygon
    ])

In [140]:
city_boundaries['box'] = city_boundaries.apply(create_polygon, axis=1)

# Convert to GeoDataFrame
city_boundaries = gpd.GeoDataFrame(city_boundaries, geometry='box')

# Set CRS to WGS84 (EPSG:4326)
city_boundaries = city_boundaries.set_crs(epsg=4326, inplace=True)

In [141]:
# Perform spatial join to keep points within polygons
emissions_cities = gpd.sjoin(df1, city_boundaries, how='inner', op='within')

/Users/maureenfonseca/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [142]:
emissions_cities.drop(columns=['source_id', 'source_name', 'source_type', 'iso3_country', 'index_right', 'bbox_north', 'bbox_south', 
                               'bbox_east', 'bbox_west', 'place_id', 'osm_type', 'lat_right', 'lon_right', 'class', 'type', 'place_rank', 'importance', 
                               'addresstype', 'name', 'display_name'], inplace=True)

In [109]:
#emissions_cities.to_csv('emissionsIPPU_pilot_cities.csv', index=False)